In [ ]:
import pandas as pd

PATH = "../data/dataset/"
DATASET = 'news_29_80k_clean_with_features_embs'

data = data.drop(["id", "channel"], axis = 1)

In [ ]:
from transformers import RobertaTokenizerFast
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", max_len=512)
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2", num_labels=29)

In [ ]:
import torch


def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('И снова даров', model, tokenizer).shape)

In [ ]:
print(embed_bert_cls(data["text"].iloc[0], model, tokenizer))

In [ ]:
import numpy as np
data['emb'] = ''

In [ ]:
data['emb'] = data["text"].apply(lambda x: embed_bert_cls(x, model, tokenizer))
data

In [ ]:
data.to_pickle("../data/dataset/news_29_80k_clean_with_features_embs.pkl")